In [1]:
# https://notebooks.azure.com/azureml/projects/azureml-getting-started/html/configuration.ipynb#Configuration
import azureml.core

print("This notebook was created using version 1.0.2 of the Azure ML SDK")
print("You are currently using version", azureml.core.VERSION, "of the Azure ML SDK")

This notebook was created using version 1.0.2 of the Azure ML SDK
You are currently using version 1.0.53 of the Azure ML SDK


In [2]:
from azureml.core import Workspace

ws = Workspace.from_config()
ws.get_details()

{'applicationInsights': '/subscriptions/165934ca-a3be-4df3-842b-332997fcecd9/resourcegroups/myresourcegroup/providers/microsoft.insights/components/llachmanmlwork7726509491',
 'containerRegistry': '/subscriptions/165934ca-a3be-4df3-842b-332997fcecd9/resourceGroups/MyResourceGroup/providers/Microsoft.ContainerRegistry/registries/llachmanmlwo5de1ae17',
 'creationTime': '2019-08-10T02:16:58.7668266+00:00',
 'description': '',
 'friendlyName': '',
 'id': '/subscriptions/165934ca-a3be-4df3-842b-332997fcecd9/resourceGroups/MyResourceGroup/providers/Microsoft.MachineLearningServices/workspaces/llachmanMLWorkspace',
 'identityPrincipalId': '2166891c-3629-4f17-8f8e-72c1b4d8d9fb',
 'identityTenantId': 'c7970e8d-6d07-4b20-bf16-eba4864e56ef',
 'identityType': 'SystemAssigned',
 'keyVault': '/subscriptions/165934ca-a3be-4df3-842b-332997fcecd9/resourcegroups/myresourcegroup/providers/microsoft.keyvault/vaults/llachmanmlwork6145098727',
 'location': 'centralus',
 'name': 'llachmanMLWorkspace',
 'stor

In [3]:
# Don't run.  This is only necessary if you are NOT using a Notebook VM
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "cpucluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Found existing cpucluster")
except ComputeTargetException:
    print("Creating new cpucluster")
    
    # Specify the configuration for the new cluster
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2",
                                                           min_nodes=0,
                                                           max_nodes=4)

    # Create the cluster with the specified name and configuration
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    
    # Wait for the cluster to complete, show the output log
    cpu_cluster.wait_for_completion(show_output=True)

Found existing cpucluster


In [4]:
# https://notebooks.azure.com/azureml/projects/azureml-getting-started/html/01.run-experiment.ipynb
from azureml.core import Experiment, Run
import math, random, pickle

In [5]:
experiment = Experiment(workspace = ws, name = "my-first-experiment")

In [6]:
# start a run to estimate Pi value
run = experiment.start_logging()

pi_counter = 0
n_iter = 100000

# Log total number of iterations
run.log("Number of iterations",n_iter)

for i in range(1,n_iter):
    # Monte Carlo step to update estimate
    x = random.random()
    y = random.random()
    if x*x + y*y < 1.0:
        pi_counter += 1
    pi_estimate = 4.0*pi_counter / i
    
    # Log convergence every 10000 iterations
    if i%10000==0:
        error = math.pi-pi_estimate
        run.log("Pi estimate",pi_estimate)
        run.log("Error",error)

# Log final results
run.log("Final estimate",pi_estimate)
run.log("Final error",math.pi-pi_estimate)

# Write file containing pi value into run history
with open("pi_estimate.txt","wb") as f:
    pickle.dump(str(pi_estimate),f)
run.upload_file(name = 'outputs/pi_estimate.txt', path_or_stream = './pi_estimate.txt')

# Complete tracking and get link to details
run.complete()
print("Run completed")

Run completed


In [7]:
# Once the run has completed, you can view a detailed report from Azure Portal by simply calling run and following the link
run

Experiment,Id,Type,Status,Details Page,Docs Page
my-first-experiment,b21d1b8b-22fa-4f8f-999e-63995d3f1d63,,Running,Link to Azure Portal,Link to Documentation


In [8]:
# You can also view all runs within an experiment. If you run the simulation above multiple times, these runs will appear 
# under the experiment view and you can compare them
experiment

Name,Workspace,Report Page,Docs Page
my-first-experiment,llachmanmlworkspace,Link to Azure Portal,Link to Documentation
